In [2]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import copy
import time
from torch.cuda.amp import GradScaler
from EarlyStopping import EarlyStopping
from Common_Function_ import *
from torch.autograd import Variable
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

GPU = '0,1,2,3'
os.environ["CUDA_VISIBLE_DEVICES"] = GPU
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
# set_seeds()

In [3]:
calculate = False
EPOCHS = 50
BATCH_SIZE = 64
VALID_RATIO = 0.3
N_IMAGES = 100
START_LR = 1e-5
END_LR = 10
NUM_ITER = 100
PATIENCE_EARLYSTOP=10

pretrained_size = 224
pretrained_means = [0.4489, 0.3352, 0.3106]#[0.485, 0.456, 0.406]
pretrained_stds= [0.2380, 0.1965, 0.1962]#[0.229, 0.224, 0.225]

In [4]:
class CustumDataset(Dataset):
    def __init__(self, data, target, data_2=None, target_2=None,transform=None):
        self.data = data
        self.target = target
        self.data_video = data_2    
        self.target_video = target_2
        self.transform = transform
        
        if self.data_video:
            self.len_data2 = len(self.data_video)
        print(self.len_data2)
        print(len(self.data_video))
        print(len(self.data))
        
        assert(self.len_data2 == len(self.target) == len(self.target_video) == len(self.data) == len(self.data_video))
    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        path = self.data[idx]
        img = Image.open(path)
        img = img.convert('RGB')
        if self.transform:
            img = self.transform(img)
            
        if self.data_video:
            path_video = self.data[idx]
            img_video = Image.open(path_video)
            img_video = img_video.convert('RGB')
            if self.transform:
                img_video = self.transform(img_video)
            return img, self.target[idx], img_video, self.target_video[idx]
        return img, self.target[idx]

In [35]:
train_transforms = transforms.Compose([
                           transforms.Resize((pretrained_size,pretrained_size)),
                           transforms.RandomHorizontalFlip(0.5),
                           # transforms.RandomCrop(pretrained_size, padding = 10),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means,
                                                std = pretrained_stds)
                       ])

test_transforms = transforms.Compose([
                           transforms.Resize((pretrained_size,pretrained_size)),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
                       ])

train_dir = ["/media/data1/mhkim/FAKEVV_hasam/train/SPECTOGRAMS/real_A_fake_B",
             "/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC"]
test_dir =  ["/media/data1/mhkim/FAKEVV_hasam/test/SPECTOGRAMS/real_A_fake_others",
             "/media/data1/mhkim/FAKEVV_hasam/test/FRAMES/real_A_fake_others"]

list_train = [datasets.ImageFolder(root = train_dir[0],transform = None),
             datasets.ImageFolder(root = train_dir[1],transform = None)]
list_test = [datasets.ImageFolder(root = test_dir[0],transform = None),
            datasets.ImageFolder(root = test_dir[1],transform = None)]


list_glob_trainpath_video = [list_train[0].samples[i][0] for i in range(len(list_train[0].samples))]
list_glob_trainpath = [list_train[1].samples[i][0] for i in range(len(list_train[1].samples))]
list_targets_trainpath_video = [list_train[0].targets[i] for i in range(len(list_train[0].targets))]
list_targets_trainpath = [list_train[1].targets[i] for i in range(len(list_train[1].targets))]

list_glob_testpath_video = [list_test[0].samples[i][0] for i in range(len(list_test[0].samples))]
list_glob_testpath = [list_test[1].samples[i][0] for i in range(len(list_test[1].samples))]
list_targets_testpath_video = [list_test[0].targets[i] for i in range(len(list_test[0].targets))]
list_targets_testpath = [list_test[1].targets[i] for i in range(len(list_test[1].targets))]
# print(list_targets_testpath[3402])
# print(list_targets_testpath_video.count(0))
# print(len(list_targets_testpath_video))

In [36]:
###################TARGET##################
cnt_target_video = list_targets_testpath_video.count(0)
cnt_target = list_targets_testpath.count(0)
temp_list = list_targets_testpath_video[:cnt_target_video]
temp_list_input = list_glob_testpath_video[:cnt_target_video]
head = cnt_target//cnt_target_video
print('head is',head)
list_0 , list_1 = {'input':[],'target':[]},{'input':[],'target':[]}
for c in range(head):
    list_0['target']+=temp_list
    list_0['input']+=temp_list_input
list_0['target']+=temp_list[:(cnt_target-len(list_0['target']))]
list_1['target']+=list_targets_testpath[cnt_target:]
list_0['input']+=temp_list_input[:(cnt_target-len(list_0['input']))]
list_1['input']+=list_glob_testpath[cnt_target:]


list_test_target_videos = list_0['target'] + list_1['target']
list_test_input_videos = list_0['input'] + list_1['input']
# print(len(list_test_target_videos))
print(list_test_target_videos==list_targets_testpath)

assert((len(list_test_target_videos))==(len(list_targets_testpath))==len(list_test_input_videos))

print(len(list_test_target_videos))
print(len(list_test_input_videos))
print(len(list_targets_testpath))

##################TRAIN####################
cnt_train_video = list_targets_trainpath_video.count(0)
cnt_train = list_targets_trainpath.count(0)
temp_list = list_targets_trainpath_video[:cnt_train_video]
temp_list_input = list_glob_trainpath_video[:cnt_train_video]
head = cnt_train//cnt_train_video
print('head is',head)
list_0 , list_1 = {'input':[],'target':[]},{'input':[],'target':[]}
for c in range(head):
    list_0['target']+=temp_list
    list_0['input']+=temp_list_input
list_0['target']+=temp_list[:(cnt_train-len(list_0['target']))]
list_1['target']+=list_targets_trainpath[cnt_train:]
list_0['input']+=temp_list_input[:(cnt_train-len(list_0['input']))]
list_1['input']+=list_glob_trainpath[cnt_train:]

list_train_target_videos = list_0['target'] + list_1['target']
list_train_input_videos = list_0['input'] + list_1['input']
print(list_train_target_videos==list_targets_trainpath)
assert((len(list_train_target_videos))==(len(list_targets_trainpath))==len(list_train_input_videos))

print(len(list_train_target_videos))
print(len(list_targets_trainpath))
print(len(list_train_input_videos))


head is 27
True
4294
4294
4294
head is 176
True
127356
127356
127356


In [38]:

import torch.utils.data as data
#image, video 한꺼번에 저장
# list_glob_trainpath = [list_train[0].samples[i][0] for i in range(len(list_train[0].samples))]
# list_glob_trainpath += [list_train[1].samples[i][0] for i in range(len(list_train[1].samples))]
# list_targets_trainpath = [list_train[0].targets[i] for i in range(len(list_train[0].targets))]
# list_targets_trainpath += [list_train[1].targets[i] for i in range(len(list_train[1].targets))]
# list_glob_testpath = [list_test[0].samples[i][0] for i in range(len(list_test[0].samples))]
# list_glob_testpath += [list_test[1].samples[i][0] for i in range(len(list_test[1].samples))]
# list_targets_testpath = [list_test[0].targets[i] for i in range(len(list_test[0].targets))]
# list_targets_testpath += [list_test[1].targets[i] for i in range(len(list_test[1].targets))]

###########################
# train_data_video = CustumDataset(list_glob_trainpath_video, list_targets_trainpath_video, test_transforms)

# n_valid_examples_video = int(len(train_data_video) * 0.2)
# n_train_examples_video = len(train_data_video) - n_valid_examples_video
# test_data_video = CustumDataset(list_glob_testpath_video, list_targets_testpath_video, test_transforms)

train_data = CustumDataset(list_glob_trainpath, list_targets_trainpath, list_train_input_videos, list_train_target_videos,test_transforms)
n_valid_examples = int(len(train_data) * VALID_RATIO)#기존 test data자체가 너무 적어서 train기준으로 비율조정
n_train_examples = len(train_data) - n_valid_examples

train_data, valid_data = data.random_split(train_data, [n_train_examples, n_valid_examples])
# train_data_video, valid_data_video = data.random_split(train_data_video, [n_train_examples_video, n_valid_examples_video])

train_data.transform = train_transforms; train_data.dataset.transform = train_transforms
valid_data.transform = test_transforms; valid_data.dataset.transform = test_transforms

test_data = CustumDataset(list_glob_testpath, list_targets_testpath, list_test_input_videos, list_test_target_videos, test_transforms)
print(len(list_glob_testpath_video), len(list_targets_testpath_video))
print(len(list_test_input_videos), len(list_test_target_videos))
from PIL import Image
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)
valid_iterator = data.DataLoader(valid_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)
test_iterator = data.DataLoader(test_data, 
                                shuffle = True, 
                                batch_size = BATCH_SIZE)

127356
127356
127356
4294
4294
4294
139 139
4294 4294
Number of training examples: 89150
Number of validation examples: 38206
Number of testing examples: 4294


### predict list중 max인 친구는 1로 바꾸고 모두 0으로 처리

In [134]:
import torch
torch.manual_seed (1414)
t = torch.randn (8, 4)
a = t.argmax (1)
m = torch.zeros (t.shape).scatter (1, a.unsqueeze (1), 1.0)
print ('\n', t, '\n\n', a, '\n\n', m)


 tensor([[ 0.1868,  1.2265,  1.0181,  0.6943],
        [ 0.1075, -0.3540,  0.1766, -0.4940],
        [ 3.0013,  1.8697, -0.0673, -1.3875],
        [-0.0560, -0.6007, -0.0410, -0.6681],
        [-0.2963,  0.1108,  0.2250, -0.5483],
        [-0.1541, -1.1390,  0.4984, -0.4016],
        [-0.3887, -0.1821,  0.4817,  0.4268],
        [ 0.8756,  2.0755,  0.2577,  0.8941]]) 

 tensor([1, 2, 0, 2, 2, 2, 2, 1]) 

 tensor([[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.]])


### EXAMPLE on Practical

In [2]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
enc = OneHotEncoder(sparse=False)

ecls.eval()
for i, data in enumerate(test_iterator):
    with torch.no_grad():
        in_1 = in_1.to(device)
        in_2 = in_2.to(device)
        print(f'{in_1.size()}, {in_2.size()}')
        y_pred = ecls(in_1,in_2).cpu().detach()
        
        in_1 = torch.tensor(data[0].detach().cpu())
        in_2 = torch.tensor(data[2].detach().cpu())
        integer_encoded = label_encoder.fit_transform(data[1].detach().cpu())
        integer_encoded_2 = label_encoder.fit_transform(data[3].detach().cpu())
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        integer_encoded_2 = integer_encoded_2.reshape(len(integer_encoded_2), 1)
        
        onehot_encoded = enc.fit_transform(integer_encoded)
        onehot_encoded_2 = enc.fit_transform(integer_encoded_2)
        onehot_encoded =onehot_encoded.astype(np.int8) 
        onehot_encoded_2 =onehot_encoded_2.astype(np.int8) 
        
        y_true = torch.tensor(onehot_encoded + onehot_encoded_2)
        y_true_argmax = y_true.argmax(1)
        y_true = np.array(torch.zeros(y_true.shape).scatter(1, y_true_argmax.unsqueeze(1),1),dtype=np.int8)
        print(y_true)

        a = y_pred.argmax (1)
        y_pred = torch.zeros (y_pred.shape).scatter (1, a.unsqueeze(1), 1)
        result = classification_report(y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=5, output_dict=False, zero_division='warn')
        print(result)
        break



NameError: name 'ecls' is not defined

In [156]:
print(result)
print(type(result))

              precision    recall  f1-score   support

           0    0.72727   0.88889   0.80000        27
           1    0.90323   0.75676   0.82353        37

   micro avg    0.81250   0.81250   0.81250        64
   macro avg    0.81525   0.82282   0.81176        64
weighted avg    0.82900   0.81250   0.81360        64
 samples avg    0.81250   0.81250   0.81250        64

<class 'str'>


In [108]:
from models.MesoNet4_forEnsemble import MesoInception4 as MesoInception4
print(f'number of train/val/test loader : {len(train_iterator), len(valid_iterator), len(test_iterator)}')
models = [MesoInception4(num_classes=2), MesoInception4(num_classes=2)]
MODELS_NAME = 'MesoInception4'
#checkpoinsts for model loaders :  [VIDEO(A&B), FRAME(A&C)]
list_checkpoint = [torch.load(f'/home/mhkim/DFVV/PRETRAINING/{MODELS_NAME}_realA_fakeB.pt')['state_dict'],
                   torch.load(f'/home/mhkim/DFVV/PRETRAINING/{MODELS_NAME}_realA_fakeC.pt')['state_dict']]
models[0].load_state_dict(list_checkpoint[0])
models[1].load_state_dict(list_checkpoint[1])
OUTPUT_DIM = 2
criterion = nn.CrossEntropyLoss().to(device)
scaler = GradScaler()
early_stopping = EarlyStopping(patience=PATIENCE_EARLYSTOP, verbose=True)

# from torchsummary import summary
# summary(models[0].cuda(), (3,224,224))

number of train/val/test loader : (1393, 597, 68)


In [131]:
class Ensemble(nn.Module):
    def __init__(self,models,device,training=True):
        super().__init__()
        self.model1 = models[0] #FOR VIDEO
        self.model2 = models[1] #FOR FRAME IMAGE
        self.fc1 = nn.Linear(16 ,224)#.to(device)
        self.fc2 = nn.Linear(16, 224)#.to(device)
        self.fc = nn.Linear(224,2)
#         selr.drop 
    def forward(self, frame, video):
        feat1 = self.model1(video)
        feat2 = self.model2(frame)
        inp1 = feat1.view(feat1.size(0), -1)
        inp2 = feat2.view(feat2.size(0), -1)
        out1 = self.fc1(inp1)
        out2 = self.fc2(inp2)
        out = out1 + out2
        out = out/2
        out = self.fc(out)
        return out
    
ecls = Ensemble(models,device)
ecls = nn.DataParallel(ecls)   # 4개의 GPU를 이용할 경우
ecls = ecls.to(device)

In [45]:
optimizer = optim.Adam(ecls.parameters(), lr = START_LR)
scaler = GradScaler()
STEPS_PER_EPOCH = len(train_iterator)
TOTAL_STEPS = EPOCHS * STEPS_PER_EPOCH
best_valid_loss = float('inf')

In [46]:
import copy
def train_voting(model, iterator, optimizer, criterion, scaler, device):
    epoch_loss = 0
    output=[]
    model.train()
    correct, total = 0,0
    
    for (x, y, x_v, y_v) in tqdm(iterator):
        with autocast(enabled=True):
            x, y, x_v, y_v = x.to(device), y.to(device), x_v.to(device), y_v.to(device)
            optimizer.zero_grad()
            final_output = model(x, x_v)
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
            
            scaler.scale(loss).backward(retain_graph=True)
            scaler.step(optimizer)
            scaler.update()

            _, predicted = torch.max(final_output.data, 1)
            correct += predicted.eq(y.data).cpu().sum()
            total += y.size(0)
        
            epoch_loss += loss.item()
    print("Accuracy = {}".format(100. * correct / total))

    epoch_loss /= len(iterator)
    epoch_acc = correct / total 
    return epoch_loss, epoch_acc


def evaluate_voting(model, iterator, criterion, device):
    epoch_loss = 0
    model.eval()
    correct, total = 0,0
    with torch.no_grad():
        for (x, y, x_v, y_v) in tqdm(iterator):
            x, y, x_v, y_v = x.to(device), y.to(device), x_v.to(device), y_v.to(device)
            final_output = model(x, x_v)
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
            
            _, predicted = torch.max(final_output.data, 1)
            correct += predicted.eq(y.data).cpu().sum()
            total += y.size(0)
            epoch_loss += loss.item()
    epoch_loss /= len(iterator)
    epoch_acc = correct / total

    return epoch_loss, epoch_acc

In [18]:
print("training...")
for epoch in range(EPOCHS):
    start_time = time.monotonic()
    train_loss, train_acc = train_voting(ecls, train_iterator, optimizer, criterion, scaler, device)
    valid_loss, valid_acc = evaluate_voting(ecls, valid_iterator, criterion, device)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save({'state_dict':ecls.state_dict(),
                   'best_acc':valid_acc,
                   'val_loss':valid_loss,
                   'epoch':epoch,
                   'lr':START_LR,
                   'best_acc':valid_acc,
                   }, f'[SFVoting,{MODELS_NAME}]with_pretrained.pt')

    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc @1: {train_acc*100:6.2f}% | ')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc @1: {valid_acc*100:6.2f}% | ')
    
    if early_stopping:
        early_stopping(valid_loss, ecls)
    if early_stopping.early_stop:
        print("Early stopping")
        break
    

training...


10


Accuracy = 71.549072265625


10


Epoch: 01 | Epoch Time: 11m 14s
	Train Loss: 0.520 | Train Acc @1:  71.55% | 
	Valid Loss: 0.440 | Valid Acc @1:  79.09% | 
Validation loss decreased (inf --> 0.439572).  Saving model ...


...

Epoch: 10 | Epoch Time: 9m 31s
	Train Loss: 0.218 | Train Acc @1:  91.20% | 
	Valid Loss: 1.043 | Valid Acc @1:  59.28% | 
EarlyStopping counter: 9 out of 10


100%|███████████| 1393/1393 [07:06<00:00,  3.27it/s]


Accuracy = 91.84745025634766


100%|█████████████| 597/597 [02:42<00:00,  3.67it/s]

Epoch: 11 | Epoch Time: 9m 49s
	Train Loss: 0.206 | Train Acc @1:  91.85% | 
	Valid Loss: 0.915 | Valid Acc @1:  61.17% | 
EarlyStopping counter: 10 out of 10
Early stopping


trn_feat_loader, val_feat_loader, test_feat_loader = Get_loader_feats(models,train_iterator,valid_iterator,test_iterator,device)

from sklearn import model_selection
from PIL import Image
from tqdm import tqdm
labels = ['model1', 'model2']
device= 'cuda'
ecls.train()
for i, (x,y) in enumerate(tqdm(train_iterator)):
    print(y)
    
    with autocast(enabled=True):
        x = x.to(device); y= y.to(device)
        output = []
        optimizer.zero_grad()
        for clf, label in zip([models[0], models[1]], labels):
            output.append(clf.module.features(x))
        print(type(output))
        final_output = ecls(output[0],output[1])
        print(final_output)
        loss = torch.FloatTensor([0.]).to(device)
        loss += criterion(final_output,y)


import copy
def train_voting(model, iterator, optimizer, criterion, scaler, device):
    epoch_loss = 0
    output=[]
    model.train()
    correct, total = 0,0
    
    for (x, y) in tqdm(iterator):
        with autocast(enabled=True):
            x, y = x.to(device), y.to(device)
#             _inp1, _inp2 = x, copy.deepcopy(x)
            optimizer.zero_grad()
            final_output = model(x)
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
            total += y.size(0)
            
            acc_1, acc_5 = calculate_topk_accuracy(final_output, y)
            scaler.scale(loss).backward(retain_graph=True)
            scaler.step(optimizer)
            scaler.update()

            _, predicted = torch.max(final_output.data, 1)
            correct += float(predicted.eq(y.data).cpu().sum())
            total += float(y.size(0))
        
            epoch_loss += loss.item()

    print("Accuracy = {}".format(100. * correct / total))

    epoch_loss /= len(iterator)
    epoch_acc = correct / total * 100
    return epoch_loss, epoch_acc


def evaluate_voting(model, iterator, criterion, device):
    epoch_loss = 0
    model.eval()
    correct, total = 0,0
    with torch.no_grad():
        for (x, y) in tqdm(iterator):
            x, y = x.to(device), y.to(device)
            final_output = model(x)
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
            
            _, predicted = torch.max(final_output.data, 1)
            correct += float(predicted.eq(y.data).cpu().sum())
            total += float(y.size(0))
            epoch_loss += loss.item()

    epoch_loss /= len(iterator)
    epoch_acc = correct / total * 100

    return epoch_loss, epoch_acc


def train_voting(model, iterator, optimizer, criterion, scaler, device):
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0
    labels = ['model1', 'model2']
    output=[]
    model.train()
    tot = 0
    correct = 0
    for (x, y) in tqdm(iterator):
        with autocast(enabled=True):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            for clf, label in zip([models[0], models[1]], labels):
                output.append(clf.module.features(Variable(x)))
            final_output = ecls(output[0],output[1])
            loss = torch.FloatTensor([0.]).to(device)
            loss += criterion(final_output,y)
#             print(loss)
            tot += y.size(0)
            
            acc_1, acc_5 = calculate_topk_accuracy(final_output, y)
            scaler.scale(loss).backward(retain_graph=True)
            scaler.step(optimizer)
            scaler.update()

            _, predicted = final_output.max(1)
            correct += predicted.eq(y).sum().item()
            epoch_loss += loss.item()
            epoch_acc_1 += acc_1.item()
    print("Accuracy = {}".format(100. * correct / tot))

    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)

    return epoch_loss, epoch_acc_1, epoch_acc_5


def evaluate_voting(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0

    model.eval()
    with torch.no_grad():
        for (x, y) in tqdm(iterator):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)

            acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc_1 += acc_1.item()
            epoch_acc_5 += acc_5.item()

    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)

    return epoch_loss, epoch_acc_1, epoch_acc_5